In [1]:
%pip install beautifulsoup4
%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# GLOBAL variable
# seed_url = ''
frontier_q = ['https://www.wongnai.com/','https://food.grab.com/th/en/','https://www.foodpanda.co.th/th/','https://lineman.line.me/','https://www.1112delivery.com/','https://www.7eleven.co.th/','https://www.robinhood.in.th/','https://www.siammakro.co.th/index.php','https://www.lotuss.com/th','https://www.tops.co.th/th/','https://cpfreshmartshop.com/']
visited_q = []
visited_host = set()
# KU_DOMAIN = 'ku.ac.th'
KEY_WORD = ('วัตถุดิบ','แคลอรี่','อาหาร','เมนู','ของกิน','กับข้าว','รสชาติ','อร่อย','เครื่องเคียง','ของว่าง','เครื่องดื่ม','ขนม')
PAGE_COUNT = 0
WEB_TYPE = ('html')
FILE_TYPE = ('pdf', 'doc', 'xls', 'ppt', 'exe', 'jpg', 'mpg', 'zip', 'xlsx', 'png')
headers = {
    'User-Agent': 'Atipat Pankong',
    'From': 'atipat.pa@ku.th'
}
debug=True

In [3]:
# check backup file
try:
    f = open('pending_queue.txt' ,'r')
    s = f.read()
    if(s):
        url_q = s.splitlines()
        PAGE_COUNT = int(url_q[0])
        frontier_q = url_q[1:]
    if(debug):
        print(PAGE_COUNT)
        print(frontier_q)
except Exception as err:
    print(err)
    print('no backup')
    PAGE_COUNT = 0
    # frontier_q = [seed_url]

0
['https://www.wongnai.com/', 'https://food.grab.com/th/en/', 'https://www.foodpanda.co.th/th/', 'https://lineman.line.me/', 'https://www.1112delivery.com/', 'https://www.7eleven.co.th/', 'https://www.robinhood.in.th/', 'https://www.siammakro.co.th/index.php', 'https://www.lotuss.com/th', 'https://www.tops.co.th/th/', 'https://cpfreshmartshop.com/']


In [4]:

# print for debug function.
def show_debug(url,error=False):
    global PAGE_COUNT
    if(not error):
        PAGE_COUNT += 1
        print('page: ',PAGE_COUNT)
    print('site: ',url)
    print('-'*50)

In [5]:
# FIFO queue
def enqueue(links):
    global frontier_q
    for link in links:
        if link not in frontier_q and link not in visited_q and not link.endswith(FILE_TYPE) and not 'mailto:' in link:
            # print('in')
            frontier_q.append(link)
        # else : 
        #     print('out')

def dequeue():
    global frontier_q
    current_url = frontier_q[0]
    frontier_q = frontier_q[1:]
    return current_url

In [6]:
import requests
from requests.exceptions import HTTPError
from urllib.parse import urljoin, urlparse

def get_page(url,require_content_type='text/html',join_set=True):
    global headers
    text = ''
    try:
        if(url.endswith(FILE_TYPE) or 'mailto:' in url): return text
        response = requests.get(url, headers=headers, timeout=2)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
        trap=0
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
        if(debug):
            show_debug(url,error=True)
    else:
        if(require_content_type in response.headers['Content-Type']):
            # response.encoding = response.apparent_encoding
            text = response.text
            if(sum([1 for x in KEY_WORD if x in text])<3):
                return ""
            if 'facebook' in url or 'youtube' in url or 'google' in url or 'instagram' in url or 'twitter' in url or 'line' in url:
                return ""
            if(join_set):
                parse = urlparse(url)
                visited_host.add(parse.scheme + '://' + parse.netloc)
        else:
            print('"' + response.headers['Content-Type'], '" is wrong content type.')
            if(debug):
                show_debug(url,error=True)
    return text.lower()

In [7]:
import os, codecs
from bs4 import BeautifulSoup
from urllib.parse import unquote

def write_html(url,html_source):
    global WEB_TYPE
    filepath = ''
    filename = 'dummy'
    result = urlparse(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    if(result.path.endswith(WEB_TYPE)):
        filepath = 'foo/' + unquote(result.netloc + result.path[:result.path.rfind('/')])
        filename = result.path[result.path.rfind('/')+1:]
    else: 
        filepath = 'html/' + unquote(result.netloc + result.path)

    os.makedirs(filepath, 0o755, exist_ok=True)
    abs_path = filepath + '/' + filename
    if(debug):
        print('write on file directory : ',abs_path)
    f = codecs.open(abs_path ,'w' ,'utf-8' )
    f.write(soup.get_text())
    f.close()

In [8]:
# parser link from html.

def absolute_link(url):
    o = urlparse(url)
    base_url = o.scheme + '://' + o.netloc
    return  base_url + o.path

def link_parser(url):
    html_source = get_page(url)
    if(html_source == ''): return []
    write_html(url,html_source)
    if(debug):
        show_debug(url)
    soup = BeautifulSoup(html_source, 'html.parser')
    all_el = soup.find_all('a',href=True)
    return [absolute_link(urljoin(url,el['href'].strip())) for el in all_el]

In [9]:
# write html > 10000 page.

while PAGE_COUNT <= 20000 and frontier_q:
    # backup url that was pending in queue.
    try:
        if PAGE_COUNT%100 == 0 :
            f = codecs.open('pending_queue.txt' ,'w' ,'utf-8' )
            f.write(str(PAGE_COUNT) + '\n'+ '\n'.join(frontier_q))
            f.close()
        current_url = dequeue()
        visited_q.append(current_url)
        enqueue(link_parser(current_url))
    except Exception as err:
        print(err)
        pass

write on file directory :  html/www.wongnai.com//dummy
page:  1
site:  https://www.wongnai.com/
--------------------------------------------------
HTTP error occurred: 403 Client Error: Forbidden for url: https://food.grab.com/th/en/
local variable 'trap' referenced before assignment
write on file directory :  html/www.7eleven.co.th//dummy
page:  2
site:  https://www.7eleven.co.th/
--------------------------------------------------
write on file directory :  html/www.robinhood.in.th//dummy
page:  3
site:  https://www.robinhood.in.th/
--------------------------------------------------
write on file directory :  html/www.tops.co.th/th//dummy
page:  4
site:  https://www.tops.co.th/th/
--------------------------------------------------
write on file directory :  html/cpfreshmartshop.com//dummy
page:  5
site:  https://cpfreshmartshop.com/
--------------------------------------------------
write on file directory :  html/www.wongnai.com/restaurants/dummy
page:  6
site:  https://www.wongnai.c

write on file directory :  html/www.wongnai.com/restaurants/shangarila/dummy
page:  23
site:  https://www.wongnai.com/restaurants/shangarila
--------------------------------------------------
write on file directory :  html/www.wongnai.com/restaurants/157461dj-ครัวดอกไม้ขาว-siam-square-one/dummy
page:  24
site:  https://www.wongnai.com/restaurants/157461dj-%e0%b8%84%e0%b8%a3%e0%b8%b1%e0%b8%a7%e0%b8%94%e0%b8%ad%e0%b8%81%e0%b9%84%e0%b8%a1%e0%b9%89%e0%b8%82%e0%b8%b2%e0%b8%a7-siam-square-one
--------------------------------------------------
write on file directory :  html/www.wongnai.com/restaurants/trattoriapizzariailbolognese/dummy
page:  25
site:  https://www.wongnai.com/restaurants/trattoriapizzariailbolognese
--------------------------------------------------
write on file directory :  html/www.wongnai.com/restaurants/440122oz-ห้างหุ้นส่วนจำกัด-88มหาสาร-ห้างหุ้นส่วนจำกัด-88มหาสาร/dummy
page:  26
site:  https://www.wongnai.com/restaurants/440122oz-%e0%b8%ab%e0%b9%89%e0%b8%b2%e0%b8%87%

KeyboardInterrupt: 

In [ ]:
# write all host in file after run 10000 page successfully.
if(debug):
  print(visited_host)
  f = codecs.open('list_host.txt' ,'w' ,'utf-8' )
  f.write('\n'.join([host for host in visited_host]))
  f.close()

In [ ]:
# find robots.txt in all host and write list of host that have robots.txt or sidemap.
# robots_host = []
# sitemap_host = []
# for host in visited_host:
#     text = get_page(host + '/robots.txt', 'text/plain',False)
#     if(text):
#         robots_host.append(urlparse(host).netloc)
#         if('sitemap:' in text):
#             sitemap_host.append(urlparse(host).netloc)

# f = codecs.open( 'list_robots.txt' , 'w', 'utf-8') 
# f.write('\n'.join(robots_host))
# f.close()

# f = codecs.open('list_sitemap.txt', 'w' , 'utf-8')
# f.write('\n'.join(sitemap_host))
# f.close()